In [2]:
from commonfunctions import  *

ImportError: cannot import name 'joblib' from 'sklearn.externals' (c:\Python311\Lib\site-packages\sklearn\externals\__init__.py)

In [ ]:
def img2gray(image: np.ndarray) -> np.ndarray:
    if len(image.shape) > 3:
        image = image[:, :, 0:3, 0]
    if len(image.shape) == 2:
        return image
    else:
        gray_img = rgb2gray(image)
        return gray_img

In [ ]:
def noiseRemoval(img: np.ndarray) -> np.ndarray:
    #clean_img = median(img, selem=disk(3))
    clean_img = gaussian(img, 1.2)
    return clean_img

In [ ]:
def median_filter(img,w=10,h=10):
    edgex = int(w/2)
    edgey = int(h/2)
    new_img = np.zeros(img.shape)
    for i in range(edgex,img.shape[0]-edgex):
        for j in range(edgey,img.shape[1]-edgey):
            new_img[i][j] = np.median(img[i-edgex:i+edgex+1,j-edgey:j+edgey+1])
    return new_img

In [ ]:
def contrastEnhancemet(img: np.ndarray) -> None:
        enhanced = img.copy()
        low, high = np.percentile(img, [0.2, 99.8])
        enhanced = rescale_intensity(img, in_range=(low, high))
        return enhanced

In [ ]:
def binarize(img: np.ndarray) -> None:
    # threshold = threshold_otsu(img)
    block_size = 25
    local_thresh = threshold_local(img, block_size, offset=0.1)
    binarized_img = (img < local_thresh).astype(np.uint8)
    return binarized_img

In [ ]:
def skewCorrection(img: np.ndarray) -> np.ndarray:
    skew_corrected_imgs = transform.resize(img, (128, 256), mode='reflect', anti_aliasing=True)
    return skew_corrected_imgs

In [ ]:
def preprocess(img: np.ndarray) -> np.ndarray:
    gray_img = img2gray(img)
    clean_img = noiseRemoval(gray_img)
    enhanced_img = contrastEnhancemet(clean_img)
    binarized_img = binarize(enhanced_img)
    #show_images([gray_img,binarized_img,opening(binarized_img,rectangle(5,5))])
    skew_corrected_img = skewCorrection(opening(binarized_img,rectangle(5,5)))
    skeletonized_img = skeletonize(skew_corrected_img)
    return skeletonized_img

In [ ]:
def readDataset(dir: str) -> list[np.ndarray]:
    dataset = []
    read_imgs = os.listdir(dir)
    for i in range(len(read_imgs)):
        image = io.imread(dir + read_imgs[i]).astype(np.uint8)
        dataset.append(preprocess(image))
    return dataset

In [ ]:
def HOGFeatureExtractionSkimage(image: np.ndarray) -> np.ndarray:
    image = image.astype(np.uint8)
    features = hog(image, orientations=9, pixels_per_cell=(9, 9),
    cells_per_block=(3,3), transform_sqrt=True, block_norm="L1")
    return features

def extractFeatures(imgs: list[np.ndarray]) -> list[np.ndarray]:
    extracted_features = []
    for i in range(len(imgs)):
        extracted_features.append(HOGFeatureExtractionSkimage(imgs[i]))
    return extracted_features


In [ ]:
real_imgs = readDataset('all/real/')
forged_imgs = readDataset('all/forged/')
test_imgs = readDataset('testImgs/')

In [ ]:
real_extracted_features = extractFeatures(real_imgs)
forged_extracted_features = extractFeatures(forged_imgs)
test_extraced_features = extractFeatures(test_imgs)

In [ ]:
HOG_features = forged_extracted_features + real_extracted_features
HOG_labels_forged = [0 for _ in range(len(forged_extracted_features))]
HOG_labels_real = [1 for _ in range(len(real_extracted_features))]
HOG_labels = HOG_labels_forged + HOG_labels_real
test_imgs_label = 1

x_train, x_test, y_train, y_test = train_test_split(HOG_features, HOG_labels, test_size = 0.2, random_state = 42)
model = svm.SVC(kernel='linear')
model.fit(x_train, y_train)

accuracy = model.score(x_test, y_test)
print("SVM using HOG as feature descriptor.", 'accuracy:', accuracy * 100, '%')
accuracy = model.score(x_train, y_train)
print("SVM using HOG as feature descriptor.", 'accuracy:', accuracy * 100, '%')

dump(model, 'model.pkl')

In [ ]:

test_prediction = model.predict(test_extraced_features)
print(test_prediction)
if test_prediction == test_imgs_label:
    print("correct prediction")
else:
    print("false prediction")